In [ ]:
# Note use the kernal, Python 3.11.7 which should be the base with all the packages installed via the terminal 
import os, json
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
# Go up one level (..) to reach course-database, then into dvc_scraper/
db_path = Path.cwd().parent / "dvc_scraper" / "Full_STEM_DataBase.json"

if not db_path.exists():
    raise FileNotFoundError(f"❌ Could not find database at: {db_path}")

with open(db_path, "r", encoding="utf-8") as f:
    course_data = json.load(f)

print(f"✅ Loaded {len(course_data)} courses from {db_path}")

✅ Loaded 109 courses from /Users/bryan/Desktop/School/Skyrelis/course-database/dvc_scraper/Full_STEM_DataBase.json


In [10]:
def search_courses(keyword: str, mode=None, status=None):
    """Return courses filtered by code/title, and optionally by format/status."""
    keyword = keyword.lower()
    results = []
    for course in course_data:
        if keyword not in course["course_code"].lower() and keyword not in course["course_title"].lower():
            continue
        filtered_sections = []
        for section in course["sections"]:
            fmt = section["meetings"][0]["format"].lower() if section.get("meetings") else ""
            stat = section["status"].lower()
            if (not mode or mode in fmt) and (not status or status in stat):
                filtered_sections.append(section)
        if filtered_sections:
            results.append({
                "course_code": course["course_code"],
                "course_title": course["course_title"],
                "sections": filtered_sections
            })
    return results
def ask_course_assistant(user_query: str):
    """Send user query to GPT for a summary."""
    # Extract course code
    keyword = ""
    for word in user_query.split():
        if "-" in word and any(ch.isdigit() for ch in word):
            keyword = word.strip(",.?!")
            break

    # ✅ Detect filters from natural language
    mode = None
    status = None
    if "in-person" in user_query.lower():
        mode = "in-person"
    elif "online" in user_query.lower():
        mode = "online"
    elif "hybrid" in user_query.lower():
        mode = "hybrid"
    if "open" in user_query.lower():
        status = "open"

    # Call updated search with filters
    results = search_courses(keyword, mode, status)
    context = (
        f"I found {len(results)} matching course(s) for '{keyword}'.\n\n"
        f"Here is the JSON data (truncated):\n{json.dumps(results, indent=2)[:4000]}"
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """You are an expert Diablo Valley College (DVC) course database interface.
Your job is to match a student's request to the available courses in the assistant's data and output *all available details* for every matching section.

Core Directive:
1. The 'user' role contains the student's request (e.g., "Find in-person COMSC courses").
2. The 'assistant' role contains JSON or text data listing all available courses.
3. Match requests semantically (e.g., "science class" → BIOSC or CHEM). Extract every section's full details.

Rules:
1. **Strict Grounding:** Only use information found in the assistant message. Do not invent any courses or details.
2. **Comprehensive Extraction:** Do not summarize. List section number, title, instructor, days, time, room, modality, units, and notes.
3. **Clarity:** Use bullet points and consistent labeling for each field.
4. **Failure Handling:**
   - If no matches: reply exactly with "No courses found matching your criteria."
   - If request is vague: reply exactly with "To help you find a course, could you please be more specific about what subject you're interested in?"

Output Format Example:

Here are the matching courses I found:

**COMSC-110: Introduction to Programming (C++)**
(This matches your request for 'COMSC course')

- Section 2045  
  - Instructor: A. Turing  
  - Days: Tue/Thu  
  - Time: 9:30 AM – 11:00 AM  
  - Location: CS 301  
  - Modality: In-Person  
  - Units: 4.0  
  - Notes: Recommended for majors.

- Section 2046  
  - Instructor: G. Hopper  
  - Days: Mon/Wed  
  - Time: 1:00 PM – 2:30 PM  
  - Location: Online  
  - Modality: Online (Synchronous)  
  - Units: 4.0  
  - Notes: None
"""
            },
            {"role": "user", "content": user_query},
            {"role": "assistant", "content": context},
        ],
    )

    return response.choices[0].message.content.strip()


In [ ]:
test_queries = [
    "Show me all COMSC-110 in-person sections for Spring 2026",
    "Find open online COMSC-165 classes available right now",
    "List every COMSC-210 section taught by Professor Stalica, including meeting times and rooms",
    "What engineering or physics courses are available in hybrid or in-person format for next term?",
    "I want a fun STEM class"
]

for q in test_queries:
    print(f"🧩 Query: {q}")
    print(ask_course_assistant(q))
    print("\n" + "-"*80 + "\n")

🧩 Query: Show me BIOSC-101 in-person sections for Spring 2026
Here are the matching courses I found:

**BIOSC-101: Fundamentals of Biological Science**
(This matches your request for 'BIOSC-101 in-person sections')

- Section 1061  
  - Instructor: Staff, DVC  
  - Days: T Th  
  - Time: 12:45 PM – 2:10 PM  
  - Location: Room 223  
  - Modality: In-Person  
  - Units: 3.00  
  - Notes: Open

- Section 1060  
  - Instructor: Preble, Mackenzie  
  - Days: M W  
  - Time: 11:10 AM – 12:35 PM  
  - Location: Room 223  
  - Modality: In-Person  
  - Units: 3.00  
  - Notes: Open

--------------------------------------------------------------------------------

🧩 Query: Find open online BIOSC-102 classes
No courses found matching your criteria.

--------------------------------------------------------------------------------

🧩 Query: List BIOSC-101 sections with instructor Preble
Here are the matching courses I found:

**BIOSC-101: Fundamentals of Biological Science**
(This matches your re

KeyboardInterrupt: 